
<H1> GET RID OF PASS IN ANALASYS FUNCTIONS</H1>

<b>Missions left:
Normalization, PCA dim reductionResampaling, Clustering, modeling, model-assesments
</b>

Eyal:
1. We treated feature 10 on sum parts as categorical and on others as numeric, we need to check again what we say and do about this feature in each part of the notebook

Alon:
1. what to do with junk @: def fill_categorical_nulls?

Daniel:
1. Convert_to_numerical feature

Daniel & Eyal:
1. Take care of sampling (check what is the threshold)
2. Feature Selection
3. 

Notes:

2. Feature 0 Discussion is required after all the plots

5. test categorical remove outliers, use most_freq from train or test?

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


<hr>

<center>
    <h2> Introduction to Machine Learning course </h2>
    <h1>Course Final Project</h1>
    <h6>By Group 21: Eyal Apelboim (302820832), Alon Regensteiner (205419716), Daniel Levkovitz (203321740)</h6>
</center>

<hr>

<h2>Imports</h2>

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = None  # visual dataframe prints configuration

from statsmodels.graphics.gofplots import qqplot

import seaborn as sns  # for graph output
from scipy import stats  # for z-score
from scipy import interp  

import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120  # setting default dpi value to all plots
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D 
plt.style.use('ggplot')
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.cluster import KMeans
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import silhouette_score, pairwise_distances, roc_curve
from sklearn.impute import SimpleImputer

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


# Display more rows in df head
pd.options.display.max_columns = None

<hr>

<h1>0. Analysis Functions</h1>
This section contains various function we've build to analyze the data later on in the next sections.

In [ ]:
def analyze_label(label):
    return
    """
    performing analysis for the data label
    """
    des = label.describe()
    rows = des['count']
    mean_val = des['mean']
    print(f'Out of {int(rows)} samples, {int(mean_val * rows)} ({str(mean_val * 100)[:5]}%) are labeled 1.')
    print(f'And the rest ({str((1 - mean_val) * 100)[:5]}%) are labled 0.')
    
    # Creating pie chart for label values
    names = ["Label=1", "Label=0"]
    values = [mean_val * rows, (1 - mean_val) * rows]
    null_pr = pd.DataFrame({"names": names, "values": values}).groupby("names")["values"].sum()
    pie, ax = plt.subplots(figsize=[3,3])
    plt.pie(x=null_pr, autopct="%.2f%%", explode=[0.05]*2, labels=null_pr.keys(), pctdistance=0.5, colors= ["lightgrey", "dodgerblue"])
    plt.title(f"Labels")
    plt.show()
    plt.clf()
    return mean_val

In [ ]:
def analyze_numerical_feature(dataframe, feature, bins=50):
    return
    """
    general function which performs full analysis for numerical freature
    """
    df = dataframe.copy()
    f = df[feature].copy()
    # getting general information
    print('Feature ' + feature + ' analysis:')
    print('-----------------------')
    print(f.describe())
    print('-----------------------')
    print("\nNulls count:", f.isna().sum())
    print('\nNull cells precentage: ' + str(f.isna().sum() * 100 / df.shape[0])[:6], '%')
    print('-----------------------')
    # Creating pie chart for Nulls & Non-Nulls
    names = ["Nulls", "Non-Nulls"]
    values = [f.isna().sum(), df.shape[0] - f.isna().sum()]
    null_pr = pd.DataFrame({"names": names, "values": values}).groupby("names")["values"].sum()
    pie, ax = plt.subplots(figsize=[3,3])
    plt.pie(x=null_pr, autopct="%.2f%%", explode=[0.05]*2, labels=null_pr.keys(), pctdistance=0.5, colors= ["lightgrey", "crimson"])
    plt.title(f"Feature {feature} Nulls Precentage");
    plt.show()
    plt.clf()
    
    # creating histogram plot
    plt.figure(figsize=(8, 24 * 4))
    gs = gridspec.GridSpec(28, 1)
    ax = plt.subplot(gs[1])
    sns.distplot(f, bins=bins, label='All')
    ax.set_xlabel('Value')
    ax.set_title(f'Feature {feature} Distribution Histogram')
    ax.legend()
    # ax.set_ylim(0, 1.0)
    plt.show()  # plotting graph
    plt.clf()  # releasing plt
    
    # creating histogram plot with separation for label 0 and label 1
    plt.figure(figsize=(8, 24 * 4))
    gs = gridspec.GridSpec(28, 1)
    ax = plt.subplot(gs[1])
    sns.distplot(f[df.label == 1], bins=bins, label='label=1', color='darkorange')
    sns.distplot(f[df.label == 0], bins=bins, label='label=0', color='dodgerblue')
    ax.set_title(f'Feature {feature} Label Distribution Histogram')
    ax.set_xlabel('Value')
    ax.legend()
    # ax.set_ylim(0, 1.0)
    plt.show()  # plotting graph
    plt.clf()  # releasing plt
    
    # creating box plot  
    fig = plt.figure(figsize=(8, 1))
    ax = fig.add_subplot(1, 1, 1)
    ax.boxplot(list(f.dropna()), vert=False)
    ax.set_title(f'Feature {feature} Boxplot graph')
    ax.set_xlabel('Value')
    ax.set_ylabel(' ')
    plt.show()  # plotting graph
    plt.clf()  # releasing plt
    
    # creating plot - probability of label 1
    edges = f.describe(percentiles = [.02, .98])  # getting feature's edge values
    bins = np.linspace(edges['2%'], edges['98%'], (15))
    bins = list(np.around(np.array(bins), 1))
    labels = bins[:-1]
    rpt = (df.groupby(pd.cut(f, bins, labels=labels)).label.mean()).to_frame('label')
    rpt.plot.bar(figsize=(8, 3), ylim=(0,1), title=f'Feature {feature} Probability of label=1 for each value', xlabel='Value', ylabel='Probability')
    # adding mean label value from all features as a benchmark:
    plt.axhline(y=label_bm, label =r"$\mu_{all} = $" + str(label_bm)[:5], color='dodgerblue', linewidth=2.0)
    plt.legend(loc ='upper right')  # showing legend
    plt.show()
    plt.clf()
    
    # creating QQ plot to check normal distribution
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title(f'Feature {feature} QQplot')
    qqplot(f.dropna(), line='s', ax=ax)  # plotting graph


In [ ]:
def analyze_categorical_feature(dataframe, feature):
    return
    """
    general function which performs full analysis for categorical freature
    """
    df = dataframe.copy()
    f = df[feature].copy()
    # getting general information
    print('Feature ' + feature + ' analysis:')
    print('-----------------------')
    print(f.describe())
    print('-----------------------')
    print("\nNulls count:", f.isna().sum())
    print('\nNull cells precentage: ' + str(f.isna().sum() * 100 / df.shape[0])[:6], '%')
    
    # Creating pie chart for Nulls & Non-Nulls
    names = ["Nulls", "Non-Nulls"]
    values = [f.isna().sum(), df.shape[0] - f.isna().sum()]
    null_pr = pd.DataFrame({"names": names, "values": values}).groupby("names")["values"].sum()
    pie, ax = plt.subplots(figsize=[3,3])
    plt.pie(x=null_pr, autopct="%.2f%%", explode=[0.05]*2, labels=null_pr.keys(), pctdistance=0.5, colors= ["lightgrey", "crimson"])
    plt.title(f"Feature {feature} Nulls Precentage");
    plt.show()
    plt.clf()
    
    # Plotting categorial feature:
    count = f.count()
    print('\nCategories Count:')
    print(f.value_counts())
    vc = f.value_counts()
    counts = vc.values.tolist()
    names = vc.index.tolist()
    plt.figure(figsize=(8, 3))
    plt.title(f'Feature {feature} Distribution')
    splot = sns.barplot(x=names, y=counts, data=vc.reset_index())
    for p in splot.patches:
        splot.annotate(format(p.get_height() * 100 / count, '.2f') + '%',
                       (p.get_x() + p.get_width() / 2., p.get_height()),
                       ha = 'center', va = 'center', 
                       xytext = (0, 9),
                       textcoords = 'offset points')
    plt.xlabel('Category')
    plt.ylabel('Count')
    plt.show()
    plt.clf()
    
    # creating plot - probability of label 1
    rpt = (df.groupby(f).label.mean()).dropna()
    rpt.plot.bar(figsize=(8, 3), ylim=(0,1), title=f'Feature {feature} Probability for label=1 per category', xlabel='Category', ylabel='Probability')
    # adding mean label value from all features as a benchmark:
    plt.axhline(y=label_bm, label =r"$\mu_{all} = $" + str(label_bm)[:5], color='dodgerblue', linewidth=2.0)
    plt.legend(loc ='upper right')  # showing legend
    plt.show()
    plt.clf()

In [ ]:
def correlation_heatmap(df):
    return
    """
    display correlation matrix graph between different columns (features and label) in df,
    using only numeric type columns
    """
    # Plot
    plt.figure(figsize=(10,8), dpi=300)
    sns.heatmap(df.corr(), xticklabels=df.corr().columns, yticklabels=df.corr().columns,
                cmap='RdYlGn', center=0, annot=True, annot_kws={"fontsize":7}, fmt='.2f')

    # Decorations
    plt.title('Correlogram of mtcars', fontsize=12)
    plt.xticks(fontsize=8)

<hr>

<h1>1. Data Exploration</h1>
Using our pre-built analysis functions (Section <i>0. Analysis Functions</i>) to perform exploration of the raw data's scale, types & more.

First, we'll have a quick look at the raw data input:

In [ ]:
# importing raw data to data frame:

#df = pd.read_csv('train.csv')  # path to raw train data file
#df_origin_test = pd.read_csv('test_without_target.csv')  # path to raw train data file
df = pd.read_csv('../input/mldata/train.csv')  # path to raw train data file
df_origin_test = pd.read_csv('../input/mldata/test_without_target.csv')  # path to raw train data file

df.head()  # showing data head:

Initial inspection of the data shows that we have 20 features of various types.

<h3>Features Overview</h3>

In [ ]:
# reviewing features types:
d_types = df.dtypes
print('Columns Types:')
print(d_types)
print('-----------------\n')
print('Total:')
print(d_types[:21].value_counts())

It seems that some of the features are categorical, mostly are numeric and we might have some "mixed types" that may require conversion to a better suited type later on.
We will conduct a full analysis to each feature.

<h3>Label</h3>
We'll take a look at the label's 1/0 distribution:

In [ ]:
label_bm = analyze_label(df['label'])  # performing analysis for the data label

Currently, before we performed any processing, it seems that we have a smaller amount of '1' labels in our input data, compared to the ideal ratio of 50:50.
It seems that there's a non-neglectible difference between labels, that perhaps we should attend later on.


<hr>

<h3>Feature 0 Analysis</h3>
According to the initial overview - the first feature is float type.
We'll use our pre-built numeric analysis function:

In [ ]:
analyze_numerical_feature(df, '0')  # using pre-built analysis function

<h4>Feature 0 Discussion:</h4>

Feature 0 presents many outliers, as viewed on most graphs above. The boxplot clearly shows it.
QQPlot displays a line in which the edges are not normal distributed, but the middle might present a normal distribution.


<hr>

<h3>Feature 1 Analysis</h3>
According to the initial overview - the second feature is object (text) type.
We'll use our pre-built categorical analysis function:

In [ ]:
analyze_categorical_feature(df, '1')  # using pre-built analysis function

<h4>Feature 1 Discussion:</h4>

Feature 1 has a very small amount of nulls. We see that value 'a' holds 2/3 of the rows. 
In this feature, we can see that the category 'unknown' takes a large portion of the data and also have some tendency for label value 0. We'll leave it as-is due to the lack of information of it's meaning.
(for example, 'Unknown' caller id might be a strong classifier for a 'scam' call).

<hr>
<h3>Feature 2 Analysis</h3>
We first start with a categorical analysis, as the feature type is 'object'.

In [ ]:
analyze_categorical_feature(df, '2')  # using pre-built analysis function

It seems this feature requires some editing in order to be able perform any analysis on it, as it's not truly categorical. It seems that all values have a redundant 'd' char next to them.
We'll create a copy of the dataframe to check if modifing the feature helps:

In [ ]:
df_test_f2 = df.copy()
df_test_f2['2'] = df_test_f2['2'].dropna().apply(lambda x: float(str(x)[:-1]))
df_test_f2.head()

In [ ]:
analyze_numerical_feature(df_test_f2, '2')  # using pre-built analysis function

<h4>Feature 2 Discussion:</h4>
The boxplot reavels that nearly every value besides '-1' is an outlier.
Looking at the graphs, we can see that this feature does not supply us any valuable information.
The QQPlot displays a straight line, which means it's abnormal.

<hr>
<h3>Feature 3 Analysis</h3>
This features seems to have numeric values. We'll use the proper tool to analyze it:

In [ ]:
analyze_numerical_feature(df, '3')  # using pre-built analysis function

<h4>Feature 3 Discussion:</h4>

Feature 3 has a very small amount of nulls.
This feature has a normal distribution behaviour according to the QQPlot, and we can see there's no difference between the lable's values, according to the label distribution. 


<hr>
<h3>Feature 4 Analysis</h3>
Seems like another numerical feature. Performing initial analysis accordingly:

In [ ]:
analyze_numerical_feature(df, '4')  # using pre-built analysis function

<h4>Feature 4 Discussion:</h4>

This feature has a negligible amount of nulls.
This feature has high denisity around the 470 value, and a large portion of outliers as seen in the boxplot graph.

<hr>
<h3>Feature 5 Analysis</h3>
Numerical as well:

In [ ]:
analyze_numerical_feature(df, '5')  # using pre-built analysis function

<h4>Feature 5 Discussion:</h4>

This feature has a negilible amount of nulls.
This feature's distribution behaves as a normal distribution, according to the qqplot, boxplot, and the distribution. In addition, we can see that the edges have a higher probability to recieve the label '1'.


<hr>
<h3>Feature 6 Analysis</h3>
Seems to be categorical. Using the proper pre-made function we've built to perform analysis:

In [ ]:
analyze_categorical_feature(df, '6')  # using pre-built analysis function

<h4>Feature 6 Discussion:</h4>
The feature has a negilible amount of nulls.
The values A D and B are most frequent, and together hold more than 50% of the rows.
We can see that different categories have different probabilites to recieve 1/0 labels.

<hr>
<h3>Feature 7 Analysis</h3>

In [ ]:
analyze_numerical_feature(df, '7')  # using pre-built analysis function

<h4>Feature 7 Discussion:</h4>

The feature has a negilible amount of nulls.
This feature seems to contribe of multiple normal distributions.
We don't see a strong correlation to specific labels.

<hr>
<h3>Feature 8 Analysis</h3>

In [ ]:
analyze_numerical_feature(df, '8')  # using pre-built analysis function

<h4>Feature 8 Discussion:</h4>
We first tried to analyze this feature as a numerical feature, but then realized it's actaully categorical.

Analyzing it, the features seems categorical. 

In [ ]:
analyze_categorical_feature(df, '8') # using pre-built analysis function

<h4>Feature 8 Discussion: (Categorical)</h4>
This feature has no nulls. The distribution of values is roughly even (only 2 values). We don't see strong correlation with the label, as both values are close to the mean of the label. However, the '0' value has stronger correlation with label '1'. 

<hr>
<h3>Feature 9 Analysis</h3>

In [ ]:
analyze_numerical_feature(df, '9')  # using pre-built analysis function

<h4>Feature 9 Discussion:</h4>
Negilible amount of nulls.
According to the qqplot and dist we can see that this feature does not match any known distribution. However,there're some outliers according to the boxplot.
Moreover, we can see a similar line for both labels in the label distribution graph, which might prevent us from learning any information from this feature.

<hr>
<h3>Feature 10 Analysis</h3>

In [ ]:
analyze_categorical_feature(df, '10')  # using pre-built analysis function

Let's try to view this feature as a numerical feature:

In [ ]:
analyze_numerical_feature(df, '10')  # using pre-built analysis function

<h4>Feature 10 Discussion:</h4>
This feature seems numeric, after both analyzes.
The values seem to have some mathematical meaning.
Most of the rows have the value '0'.
We can observe that the probabilty to get a label '1' is larger as we get farther away from value 0.

<hr>
<h3>Feature 11 Analysis</h3>


In [ ]:
analyze_numerical_feature(df, '11')  # using pre-built analysis function

<h4>Feature 11 Discussion:</h4>
Negilible amount of nulls.The distribution is somewhat similar to a normal dist.
However, we can see that the outliers prevent this feature from being normal, accodring to qqplot. We don't see any trend for the labels, which means it doesn't provide us with a lot of information.

<hr>
<h3>Feature 12 Analysis</h3>

In [ ]:
analyze_categorical_feature(df, '12')  # using pre-built analysis function

<h4>Feature 12 Discussion:</h4>
Neglible amount of nulls.
We can see we have a lot more 'N' (which probably represents 'No'), than 'Y' ('yes').
'y' has a strong correcltion to label 0.

<hr>
<h3>Feature 13 Analysis</h3>

In [ ]:
analyze_numerical_feature(df, '13')  # using pre-built analysis function

We can see by the qqplot and distributons that this is a categorical feature.

In [ ]:
analyze_categorical_feature(df, '13')  # using pre-built analysis function

<h4>Feature 13 Discussion:</h4>
We can see that value 2 is most of the rows. We don't see any strong correlation in the probabilities graph, however, 0 and 3 are better correlated with 1's, and values 1, 2 are better correlated with 0.

<hr>
<h3>Feature 14 Analysis</h3>

In [ ]:
analyze_numerical_feature(df, '14')  # using pre-built analysis function

<h4>Feature 14 Discussion:</h4>
There's a non-neglible amount of nulls, that might effect the model behaviour.
The feature distribution looks like an F distribution, which around the mean values are correlated with the label '0'.
In the label distribution graph we see a difference between the values of 0 and 1.
The probabilty to get the label '1' is stronger as the value gets larger.

<hr>
<h3>Feature 15 Analysis</h3>

In [ ]:
analyze_numerical_feature(df, '15')  # using pre-built analysis function

This feature behaves more like a categorical feature. To analyze it, we'll use the categorical analyzation:

In [ ]:
analyze_categorical_feature(df, '15')  # using pre-built analysis function

<h4>Feature 15 Discussion:</h4>
This feature doesn't give us a lot of information, because the overwhelming majority of values are '0'. 

<hr>
<h3>Feature 16 Analysis</h3>

In [ ]:
analyze_categorical_feature(df, '16')  # using pre-built analysis function

<h4>Feature 16 Discussion:</h4>
We assume this feature might represent time values (seconds, minutes, days).
We can see that 'M' is more common than S,D. However, non of which tell us a lot of informationm, because they all have a similar label distribution.

<hr>
<h3>Feature 17 Analysis</h3>

In [ ]:
analyze_numerical_feature(df, '17')  # using pre-built analysis function

<h4>Feature 17 Discussion:</h4>
This feature has an tendency to normal dist, accodring to qqplot and the distrubtion. Moreover, we see many outliers according to boxplot.
The edges tendency is to the '1' values, and the middle values have more '0' labels.

<hr>
<h3>Feature 18 Analysis</h3>

In [ ]:
analyze_categorical_feature(df, '18')  # using pre-built analysis function

<h4>Feature 18 Discussion:</h4>
This feature has about a third of it's values with the value 'a5', which correlates with the label 0.
We can see from the graph that most of the categories have some correlation with the label value.

<hr>
<h3>Feature 19 Analysis</h3>

In [ ]:
analyze_categorical_feature(df, '19')  # using pre-built analysis function

<h4>Feature 19 Discussion:</h4>
Most of the values are nulls, therefore we expect this feature will be removed later on, because it doesn't provide us with any <b> reliable</b> information.

<hr>
<h3>Feature 20 Analysis</h3>

In [ ]:
analyze_numerical_feature(df, '20')  # using pre-built analysis function

<h4>Feature 20 Discussion:</h4>
This features doesn't have any nulls. It looks like a normal distribution, without any correlation to one of the labels.


<hr>
<h3>Numeric Features Correlation Analysis</h3>
Now that we've analyzed all features, we'd like to look for correlation with the labels.

In [ ]:
df_coor_test = df.copy()  # creating a copy of the df for correlations testing
df_coor_test['2'] = df_coor_test['2'].dropna().apply(lambda x: float(str(x)[:-1]))  # updating df copy so feature 2 will be numeric
correlation_heatmap(df_coor_test)  # using your pre-built function to create a correlations graph for analysis

<h4>Numeric Features Correlation Discussion:</h4>
We can see the highest correlation with the label classification is 0.43, and is related to feature no. 14.
We can see that featurs 10,8,2,0 have some correlation with a label classification. The other features have neglectible correlation values.
In addition, we also notice that features pair (9, 7), as well as (17, 5) have a high correlation.
Feature pairs (9,3), (7,3), (10,2), have a medium correlation. The other features have either a low or a neglectible correlation. 

<hr>

<h1>2. Pre-Processing</h1>
After we've observed and studied our input data, we'll now prepare it for processing.

In [ ]:
# fresh start: reading both train and test raw files
#df = pd.read_csv('train.csv')  # path to raw train file
#df_origin_test = pd.read_csv('test_without_target.csv')  # path to raw test file
df = pd.read_csv('../input/mldata/train.csv')  # path to raw train data file
df_origin_test = pd.read_csv('../input/mldata/test_without_target.csv')  # path to raw train data file

In [ ]:
# creating copies of dfs:
df_train = df.copy()
df_test = df_origin_test.copy()
df_test = df_test.drop(df_test.columns[0],axis=1,inplace=False)  # removing raw input index column

In [ ]:
# checking that both train and test have the same amount of features (columns excluding label column)
print(len(df_train.columns.drop('label')))
print(len(df_test.columns))

In [ ]:
# displaying both sets head to compare:
df_train.head()

In [ ]:
df_test.head()

Now that we hold both data sets in the same condition, we'll start applying changes so they can fit to the processing stage:

<h3>Values Types Handling</h3>

In [ ]:
def remove_char(dataframe, feature, c):
    """
    handling feature's numeric values with combined string element c by removing c from value
    """
    dataframe[feature] = dataframe[feature].map(lambda x: x.strip(c) if type(x) == type('str') else x)
    return None

def convert_to_numerical(dataframe, feature):
    """
    updating feature's values from objects to numrics
    """
    dataframe[feature] = dataframe[feature].apply(pd.to_numeric)
    print('dtypes    ',feature, dataframe['2'].dtypes)
    return None


# Remove redundant characters from feature 2
remove_char(df_train, '2', 'd')
remove_char(df_test, '2', 'd')
convert_to_numerical(df_train, '2')
convert_to_numerical(df_test, '2')

In [ ]:
def convert_to_categorical(dataframe, feature):
    """
    changing categorial features from numeric
    """
    dataframe[feature] = dataframe[feature].astype("object")
    return None

# updating categorical features to correct type (were initially classified as numeric due to their values type):
convert_to_categorical(df_train, '8')
convert_to_categorical(df_train, '13')
convert_to_categorical(df_train, '15')
convert_to_categorical(df_test, '8')
convert_to_categorical(df_test, '13')
convert_to_categorical(df_test, '15')

In [ ]:
# checking both data sets are still aligned:
print(len(df_train.columns.drop('label')))
print(len(df_test.columns))

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
# checking for same types
df_test.dtypes == df_train.dtypes.drop('label')

<h3>Missing Data & Outliers Handling</h3>

In [ ]:
def relocate_value(dataframe, feature, val):
    """
    relocate selected value to most frequent category
    """
    most_freq = dataframe[feature].describe()['top']
    if most_freq == val:
        print('most freqent is the value itself')
        return None
    return dataframe.replace([val], most_freq)
    
def drop_nulls_rich_features(dataframe, threshold=0.5):
    """
    reviewing each feature for precentage of nulls.
    if precentage is higher than threshold - dropping the feature from dataframe.
    """
    # Drop columns based on threshold limit
    updated_df = dataframe.dropna(axis=1, thresh=len(dataframe.index) * (1 - threshold))
    return updated_df

def get_mean_wo_outliers(dataframe, feature):
    """
    getting z-score based mean from train dataframe
    """
    z = np.abs(stats.zscore(dataframe[dataframe[feature].notna()][feature]))
    mean_val = dataframe[dataframe[feature].notna()][z < 3][feature].mean()
    return mean_val

def remove_outliers(dataframe):
    """
    remove outliers from dataframe (according to z score)
    Using data deviation
    """
    dataframe = dataframe[(np.abs(stats.zscore(dataframe[dataframe.describe().columns.values[:-1]])) < 3).all(axis=1)]
    return dataframe 

def fill_numerical_nulls(dataframe, mean, feature): # perhapds add isTrain
    """
    updating selected feature nulls with values based on the rest of the data
    """
    f_type = dataframe[feature].dtype  # checking what is the type of the data to handle accordingly
    if f_type == float:
        mean_val = mean  # getting mean value to replace nulls with          
        dataframe[feature] = dataframe[feature].fillna(mean_val)
    return dataframe

def get_most_freq_imputer(dataframe, feature):
    """
    finds and returns an imputer based on most frequent value of given df
    """
    #return dataframe[feature].describe()['top']
    imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    imputer.fit(dataframe[feature].values.reshape(-1, 1))
    return imputer


def fill_categorical_nulls(dataframe, feature, imputer):
    """
    filling (replacing) nulls with the most frequent value on a certain feature
    """
    dataframe[feature] = imputer.transform(dataframe[feature].values.reshape(-1, 1))
    return dataframe

In [ ]:
# drop null rich features, default threshold is: >50% nulls
df_train = drop_nulls_rich_features(df_train)

# dropping test colums based on the ones that were dropped from the train set:
df_test = df_test[df_train.columns.drop('label')]

In [ ]:
# fill nulls for numerical
for f in df_train.describe().columns[:-1]:
    feature_mean = get_mean_wo_outliers(df_train, f)
    df_train = fill_numerical_nulls(df_train, feature_mean , f) # fill nulls according to train mean
    df_test = fill_numerical_nulls(df_test, feature_mean, f) # fill nulls according to train mean

In [ ]:
# fill nulls for categorical and one-hot encoding
for f in df_train.select_dtypes(['O']).columns:
    train_imputer = get_most_freq_imputer(df_train, f)
    df_train = fill_categorical_nulls(df_train, f, train_imputer)
    df_test = fill_categorical_nulls(df_test, f, train_imputer)

In [ ]:
# remove outliers from df (using data deviation)
df_train = remove_outliers(df_train)

In [ ]:
df_train.head()

In [ ]:
df_train.isna().sum()

In [ ]:
df_test.head()

In [ ]:
df_test.isna().sum()

<h3>Categorical Features Handling</h3>

In [ ]:
def one_hot_encoding(dataframe, categorial_feature):
    """
    transforming categorial value to 'one hot vector'
    """
    df = dataframe
    f = categorial_feature
    # create a hot vector matrix, naming the column's
    # prefix with the featrue's name
    du = pd.get_dummies(df[f],prefix = f)
    # concating new columns with exisitng dataframe
    update_df = pd.concat([df, du], axis=1)
    # deleting old column
    update_df = update_df.drop(columns=[f])
    return update_df

In [ ]:
# transforing all categorical features to one-hot vectors
for f in df_train.select_dtypes(['O']).columns:
    df_train = one_hot_encoding(df_train, f)
    df_test = one_hot_encoding(df_test, f)



In [ ]:
# Making sure train and test are still alignd, after hot-vector operations
print(df_test.columns == df_train.columns.drop('label'))

<h3>Data Scaling</h3>

In [ ]:
def get_unscaled_features(dataframe):
    '''get list of features that are not standartaized'''
    not_std_features = []
    # go over all numerical features and label
    for feature in dataframe.describe().columns:
        # Look for one hot vectors or already normalized features
        if not ((max(dataframe[feature]) == 1.0) and (min(dataframe[feature]) == 0.0)):
            not_std_features.append(feature)   
    return not_std_features

def get_fit_scaler_std(dataframe, features):
    '''get the standartization scaler'''
    scaler = StandardScaler()
    return scaler.fit(dataframe[features])

def get_fit_scaler_minmax(dataframe, features):
    '''get the minmax scaler'''
    scaler = MinMaxScaler()
    return scaler.fit(dataframe[features])

# standardize feature with scalar
def scale_features(dataframe, features, scaler):
    '''transform dataframe with scaler'''
    df_scaled = pd.DataFrame(scaler.transform(dataframe[features].values), columns=features, index=dataframe.index)
    return df_scaled

First, we'll scale only numeric features, and avoid categorical one-hot-vectors, and the label.

In [ ]:
unscaled_features = get_unscaled_features(df_train)

# create standardScaler
standardScaler_train = get_fit_scaler_std(df_train, unscaled_features)
# create minmax scaler
minmaxScaler_train = get_fit_scaler_minmax(df_train, unscaled_features)

# create copies for both scaling methods
df_train_std = df_train.copy()
df_test_std = df_test.copy()
df_train_minmax = df_train.copy()
df_test_minmax = df_test.copy()
# standartization scaling:
df_train_std[unscaled_features] = scale_features(df_train, unscaled_features, standardScaler_train) # train
df_test_std[unscaled_features] = scale_features(df_test, unscaled_features, standardScaler_train) # test using the same scaler
# minmax scaling:
df_train_minmax[unscaled_features] = scale_features(df_train, unscaled_features, minmaxScaler_train) # train
df_test_minmax[unscaled_features] = scale_features(df_test, unscaled_features, minmaxScaler_train) # test using the same scaler

In [ ]:
df_train_std.head()

In [ ]:
df_test_std.head()

In [ ]:
df_train_minmax.head()

In [ ]:
df_test_minmax.head()

<h3>Dimension Reduction</h3>
We will use the PCA method reduce the dimentions by creating new dataframe with less columns which hold 98% of the variance

In [ ]:
def get_pca(dataframe):
    pca = PCA(n_components = dataframe.shape[1])
    pca.fit(dataframe)
    dataframe_new = pca.transform(dataframe)
    explaind_variance_ratio = pca.explained_variance_ratio_
    return dataframe_new, explaind_variance_ratio

def get_number_of_features_for_variance(dataframe, exp_variance):
    pca = PCA(n_components = exp_variance, svd_solver = 'full')
    pca.fit(dataframe)
    dataframe_new = pca.transform(dataframe)
    explaind_variance_ratio = pca.explained_variance_ratio_
    return dataframe_new, explaind_variance_ratio

def get_pca_fit(dataframe, exp_variance):
    pca = PCA(n_components = exp_variance, svd_solver = 'full')
    return pca.fit(dataframe)

def pca_transformation(dataframe, pca_fitted):
    dataframe_new = pd.DataFrame(pca_fitted.transform(dataframe), index=dataframe.index)
    return dataframe_new


def accumulative_pca_graph(dataframe1, dataframe2, explained_variance_ratio1, explained_variance_ratio2):
    fig, ax = plt.subplots(figsize=(10,5))
    plt.plot(explained_variance_ratio1.cumsum(),color='navy', label="std")
    plt.plot(explained_variance_ratio2.cumsum(),color='green',linestyle="--", label="minmax")
    plt.title("Accumulative explanation by PCA", fontsize=11)
    plt.ylabel('Accumulative explained variance', fontsize=11)
    plt.xlabel('No. Of features', fontsize=11)
    plt.xticks(range(0,dataframe1.shape[1],1))   
    plt.axhline(0.98, c='r')
    plt.legend()
    plt.show()

In [ ]:
# delete the 'label'
df_train_std_no_label = df_train_std.copy()
del df_train_std_no_label['label']
df_train_std_no_label

In [ ]:
df_pca_std, variance_std = get_pca(df_train_std[unscaled_features])
df_pca_minmax, variance_minmax = get_pca(df_train_minmax[unscaled_features])
accumulative_pca_graph(df_pca_std,df_pca_minmax,variance_std, variance_minmax)

We can see that the minmax scaled dataset requires a similar amount of 9 dimentions to conserve the 98% of the variance

In [ ]:
train_pca_fit = get_pca_fit(df_train_std[unscaled_features], 0.98)
# Transform train according to TRAIN fit
train_std_pca_reduced = pca_transformation(df_train_std[unscaled_features], train_pca_fit )
# Transform test according to TRAIN fit (TRAIN, not TEST)
test_std_pca_reduced = pca_transformation(df_test_std[unscaled_features], train_pca_fit )

train_std_pca_reduced = pd.concat([train_std_pca_reduced, df_train_std.drop(unscaled_features, axis=1)], axis=1)
test_std_pca_reduced = pd.concat([test_std_pca_reduced, df_test_std.drop(unscaled_features, axis=1)], axis=1)

In [ ]:
train_std_pca_reduced

In [ ]:
train_pca_fit = get_pca_fit(df_train_std[unscaled_features], 0.98)
# Transform train according to TRAIN fit
train_minmax_pca_reduced = pca_transformation(df_train_std[unscaled_features], train_pca_fit )
# Transform test according to TRAIN fit (TRAIN, not TEST)
test_minmax_pca_reduced = pca_transformation(df_test_std[unscaled_features], train_pca_fit )

train_minmax_pca_reduced = pd.concat([train_minmax_pca_reduced, df_train_minmax.drop(unscaled_features, axis=1)], axis=1)
test_minmax_pca_reduced = pd.concat([test_minmax_pca_reduced, df_test_minmax.drop(unscaled_features, axis=1)], axis=1)

In [ ]:
train_minmax_pca_reduced

In [ ]:
test_std_pca_reduced

In [ ]:
test_minmax_pca_reduced

<hr>

<h1>3. Model Training</h1>

Now that both train and test dataset are fully proccessed, we can proceed to the model training stage. We'll start by examining the results, without any further manipulation on the data, including not balancing the label.

In [ ]:
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score


Create Train, and Test Sets. The train will be used for training. The test will only be used only for assesing the model, not for training.

In [ ]:
X = train_std_pca_reduced.drop('label', axis=1)
y = train_std_pca_reduced['label']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

In [ ]:
clf_gnb = GaussianNB()
clf_gnb.fit(X_train, y_train)
clf_gnb.score(X_test, y_test)

## KNN

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize

knnparameters =  {'n_neighbors':[3,5,11,30,50],
                  'weights':['uniform'],
                  'metric': ['euclidean', 'manhattan']}

GS_knn = GridSearchCV(KNeighborsClassifier(), knnparameters, cv=3, scoring='roc_auc')
GS_knn.fit(X_train, y_train)

print ('KNN chosen parameters (recieved best AUC): {}'.format(GS_knn.best_params_))
print ("KNN AUC score with the chosen parameters: ", GS_knn.best_score_)

In [ ]:
clf_knn = KNeighborsClassifier(**GS_knn.best_params_)
clf_knn.fit(X_train, y_train)
print('score ',clf_knn.score(X_test, y_test)) # Model Accuracy

## ANN

In [ ]:
parametersOptions = {'activation' : ["logistic", "relu"], #
                      'hidden_layer_sizes' : [(100,), # 1 large hidden layer
                                             (50, 50), # 2 medium size layers
                                             (20, 20, 10, 10, 10)], # multiple small sized layers
                      'batch_size' : [10, 50],
                      'learning_rate_init' : [0.1, 0.01], #In some of the runs we saw that the network got stuck on a local min, for this reason we enlearge the defualt momentum
                      'max_iter' : [1500]} 

GS_ann = GridSearchCV(MLPClassifier(), parametersOptions, cv=3, scoring='roc_auc')
GS_ann.fit(X_train, y_train)

print ('ANN chosen parameters (recieved best AUC): {}'.format(GS_ann.best_params_))
print ("ANN AUC score with the chosen parameters: ", GS_ann.best_score_)

In [ ]:
ann_clf = MLPClassifier(**GS_ann.best_params_, random_state=42)  # Using the best parameters
ann_clf.fit(X_train, y_train)
print('score ',ann_clf.score(X_test, y_test)) # Model Accuracy

## Random Forest

In [ ]:
# Number of trees in random forest
n_estimators = [100,300,500,800,1200] #[int(x) for x in np.linspace(start = 100, stop = 300, num = 50)]
# Maximum number of levels in tree
max_depth = [5,10,20,40]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Method of selecting samples for training each tree
bootstrap = [True, False]

parametersOptions = {'n_estimators': n_estimators,
                       'max_depth': max_depth,
                       'min_samples_split': min_samples_split,
                       }

GS_randomForest = GridSearchCV(RandomForestClassifier(), parametersOptions, cv=3, scoring='roc_auc')
GS_randomForest.fit(X_train, y_train)

print ('RandomForest chosen parameters (recieved best AUC): {}'.format(GS_randomForest.best_params_))
print ("RandomForest AUC score with the chosen parameters: ", GS_randomForest.best_score_)

In [ ]:
randomForest_clf = RandomForestClassifier(**GS_randomForest.best_params_, random_state=42)  # Using the best parameters
randomForest_clf.fit(X_train, y_train)
print('score ', randomForest_clf.score(X_test, y_test) ) # Model Accuracy

<h1>4. Models Evaluation</h1>
We'll first create an Confusion Matrix for one of our trained models:

In [ ]:
from sklearn.metrics import plot_confusion_matrix 

def create_confusion_matrix(trained_model, X, y):
    """
    given trained model and data set, applying the data set without label
    on the trained model to produce predictions.
    Then comparing these predictions with the true labels of the data set
    to produce a confusion matrix for the model.
    """
    prediction = trained_model.predict(X)
    plot_confusion_matrix(trained_model, X, y, cmap=plt.cm.Blues)
    plt.show()  

In [ ]:
def KfoldPlot(X, y, X_valid, y_valid, clf, k):
    
    #str(clf) returns the name of the classifier, we set this as the title.
    clfname = str(clf)[:str(clf).find("(")]
    #We prepare a figure, which will be filled in with some graphs inside.
    plt.figure()
    #We initialize the KFOLD, which will be iterated later.
    kf = KFold(n_splits=k, shuffle=False)
    #Since we need to interpolate data, we set an mean tpr and fpr for later porpuses.
    mean_tpr = 0.0
    mean_tpr_train = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    mean_fpr_train = np.linspace(0, 1, 100)
    i=1
    
    #We iterate over the folds.
    for train_index, test_index in kf.split(X):
        #Just printing the current fold
        print("Folding No. ", i)
        #Splitting into train and validation, based on the current fold.
        X_train, X_test = X[train_index[0]:train_index[-1]], X[test_index[0]:test_index[-1]]
        y_train, y_test = y[train_index[0]:train_index[-1]], y[test_index[0]:test_index[-1]]
        #We fit with X_train and y_train.
        clf.fit(X_train, y_train)
        
        # predict on train
        #y_train_p = clf.predict_proba(X_train)[:,1]
        # predict on test
        #y_test_p = clf.predict_proba(X_test)[:,1]
        #fpr_tr,tpr_tr,thresholds_tr=roc_curve(y_train,y_prob_tr) #get the FPR TPR for both test and train
        #fpr_te,tpr_te,thresholds_te=roc_curve(y_test,y_prob_te)
       # plt.plot(fpr_te, tpr_te, color='steelblue',linestyle='--')
        
        # The prediction we want to test against are the probability of 1 of the X_test
        prob_prediction = clf.predict_proba(X_test)[:, 1]
        # for evaluation, we'll predict on the train itself
        prob_prediction_train = clf.predict_proba(X_train)[:, 1]
        
        # Right now we can set our ROC curve on the specific fold.
        fpr, tpr, thresholds = roc_curve(y_test, prob_prediction)
        # Doing the same for the train
        fpr_train, tpr_train, thresholds_train = roc_curve(y_train, prob_prediction_train)
        
        #We interpolate the mean_tpr so all of the classifiers will be working under the same thersholds.
        mean_tpr += interp(mean_fpr, fpr, tpr)
        #We interpolate the mean_tpr so all of the classifiers will be working under the same thersholds.
        mean_tpr_train += interp(mean_fpr_train, fpr_train, tpr_train)
        
        mean_tpr[0] = 0.0
        mean_tpr_train[0] = 0.0
        #We plot the current fold with the color #D3D3D3
        plt.plot(fpr, tpr, color='#D3D3D3')
        i+=1
    plt.plot([0], [0], color='#D3D3D3', linestyle='-', label='K-folds')    
    
    
    #Right now the mean_tpr is a sum (we added each iteration, so we finally divide by the number of folds)
    mean_tpr /= k
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    
    # we do the same for the train set
    mean_tpr_train /= k
    mean_tpr_train[-1] = 1.0
    mean_auc_train = auc(mean_fpr_train, mean_tpr_train)
    diff = np.abs(mean_auc_train - mean_auc)
    
    if diff > 0.1:
        print("The model displays overfitting properties. The test-based AUC is ",
              mean_auc, "and the train-based auc is: ", mean_auc_train, ". The difference is: ", diff)
    else:
        print("The model *does not* displays overfitting properties. The test-based AUC is ",
              mean_auc, "and the train-based auc is: ", mean_auc_train, ". The difference is: ", diff)
    
    #After all of the iterations, we plot the random-guess line
    plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
    #And we plot the mean roc, the red line should be in the middle of all gray lines.
    plt.plot(mean_fpr, mean_tpr, color='red', linestyle='-', label='Mean ROC (AUC = %0.3f)' % mean_auc)
    #Setting some boundaries and adding a legend... nothing special.
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC for ' + clfname)
    plt.legend(loc="lower right")
    plt.show()
    
    return mean_auc
    

In [ ]:
create_confusion_matrix(clf_gnb, X_test, y_test)

In [ ]:
best_model = None
models  = [clf_gnb, clf_knn, ann_clf, randomForest_clf]
max_auc = 0
for model in models:
    curr_auc = KfoldPlot(X_train, y_train, X_test, y_test , model , 5)
    if curr_auc > max_auc:
        max_auc = curr_auc
        best_model = model

We choose according to the AUC metric. The highest AUC metric belongs to

In [ ]:
best_model.fit(X_train,y_train)
prediction = best_model.predict(X_test)
create_confusion_matrix(best_model, X_test, y_test)
#create_confusion_matrix(best_model, X, y)
plt.show()


<h1>5. Prediction</h1>

In [ ]:
best_model.fit(X , y)
y_pred = best_model.predict_proba(test_std_pca_reduced)
results = pd.DataFrame(pd.DataFrame(y_pred, columns = ['pred_proba', 'to_drop'])['pred_proba'])
results.to_csv('results.csv')